In [ ]:
import boto3
from botocore.exceptions import ClientError

# Initialize DynamoDB resource
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')  # Change region

# Reference your table
table = dynamodb.Table('nova-ingest-ads-queue')
col_name = "data"

try:
    # Scan the table but only return the specific column (attribute)
    response = table.scan(
        ProjectionExpression=col_name  # Replace with your attribute
    )

    # Extract values
    items = response.get('Items', [])
    values = [item.get(col_name) for item in items]

    print("Fetched values:", values)

    # Handle pagination (DynamoDB Scan can return partial results)
    while 'LastEvaluatedKey' in response:
        response = table.scan(
            ProjectionExpression=col_name,
            ExclusiveStartKey=response['LastEvaluatedKey']
        )
        items = response.get('Items', [])
        values.extend(item.get(col_name) for item in items)

    print("All values:", values)

except ClientError as e:
    print(f"Error scanning table: {e.response['Error']['Message']}")
